# 1.1 RAG란 무엇인가?  
  
## 1.1.1 RAG란?  
검색 증강 생성(Retrieval-Augmented Generation)의 약자입니다.  
단순히 학습 데이터에 의존해 답변을 생성하는 것이 아닙니다.  
외부 데이터베이스나 문서에서 관련된 정보를 검색(Retrieval)후 생성(Generation)하는 기법을 의미합니다.  
  
## 1.1.2 RAG의 필요성  
기존에 학습된 데이터만을 사용하면 다음의 항목이 문제가 됩니다.  
* 최신 정보 반영 불가  
* 학습하지 않은 도메인에 대해서 사용불가  
* 환각현상 발생  
* 시간과 비용에 대한 투자  
  
## 1.1.3 RAG 핵심 원리  
핵심적인 3가지 단계로 구성됨.  
1) 검색  
- 사용자의 질문에 가장 관련성이 높은 정보를 검색하는 단계입니다.  
- 백터 유사도를 사용하는 경우가 많습니다.  
2) 정보 증강  
- 검색된 문서를 원래 입력에 추가해서 모델에 제공하는 단계입니다.  
3) 응답 생성  
- 기존의 LLM의 인퍼런스 단계와 동일합니다.  
  
## 1.1.4 RAG 구현 방법  
RAG를 구현하는 과정은 `DB구축 -> 백터검색 -> LLM 통합` 세단계로 나눌 수 있습니다.  
그래프 DB를 사용하는 방법도 있으나. 이 책에서는 백터만을 사용합니다.  
  
**(1) 데이터베이스 구축**  
RAG의 필수적인 초기 단계입니다.  
문서를 백터(Vector)로 변환하여 백터 데이터베이스에 저장하는 단계입니다.  
RAG에 사용할 수 있는 문서들은 다음과 같습니다.  
*\[RAG에 사용가능한 문서\]*  
* .json, .csv 등의 구조화된 데이터  
* .pdf, .docs등의 문서 파일  
* 데이터베이스(sql, nosql)  
* 웹사이트 크롤링 데이터  
LLM이 더 정확하고 신뢰성 있는 답변을 생성할 수 있도록 체계적이고 구조화된 방식으로 저장하는것이 핵심입니다.  
① openai의 `text-embedding-ada-002`같은 임배딩 모델을 사용하여 텍스트를 백터로 변환하고  
② FAISS, Pinecone, Weaviate, Chroma 같은 백터DB에 저장하여 사용합니다.  
  
**(2) 백터 검색**  
기존의 키워드 검색과는 다릅니다.  
의미적 유사성을 고려하여 정밀한 검색 결과를 제공할 수 있습니다.  

**(3) LLM과 통합**  
프롬프트에 넣는것과 다르지 않습니다.  
하지만, langchain 프레임웍은 이들을 잘 추상화 했습니다.

In [ ]:
from dotenv import load_dotenv
import os
load_dotenv()

from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_openai.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

# PDF 로드
loader = PyPDFLoader("data/succulent.pdf")
documents = loader.load()

# 백터 DB 구성
vectorstore = FAISS.from_documents(documents, OpenAIEmbeddings())

# LLM 모델 로드
llm = ChatOpenAI(model_name="gpt-4o-mini")

# 검색을 위한 QA 체인 생성
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectorstore.as_retriever()
)

# 쿼리
query = "우리회사의 최신 제품"
response = qa_chain.invoke({'query':query})
print(response.get('result'))

# 1.2 VectorRAG란 무엇인가?  
  
## 1.2.1 벡터란?
RAG 검색과 과정에서 텍스트를 벡터로 변환하는 이유는 더욱 정교하게 의미적으로 유사한 문서를 검색하기 위해서 입니다.  
기존의 키워드 기반 검색방식 TF-IDF, BM25는 단순히 문서 내 키워드의 빈도수를 활용하여 검색하기에 시멘팅 정보를 인식하지 못ㅎ바니다.  

## 1.2.2 벡터 처리 과정  
**(1) 텍스트를 벡터로 변환(임베딩)**  
임베딩 모델을 이용해서 변환합니다.  
대표적인 모델로 `text-embedding-ada-002`, `BERT`, `Sentence-BERT`등이 있습니다.  
책에서는 openai의 `text-embedding-ada-002`를 사용합니다.  
  
**(2) 벡터 검색**  
백터 검색\(Vector Retrieval\)은 사용자의 Query를 백터화한 후, DB에 저장된 백터들과 비교하여 가장 가까운 문서를 검색하는 과정입니다.  
가장 가까운 문서를 찾기 위해서 널리 사용되는 방법은 유클리드 거리와 코사인 유사도입니다.  

<유클리드 거리>  
백터의 크기를 고려하여 검색  
차원의 저주가 발생가능 함  
위치기반 추천, 이미지 검색등에 사용됨  

<코사인 유사도>  
두 백터간의 각도만 고려함  
고차원에서도 유클리드에 비해서 안정적임  
문서 검색, NLP, 추천 시스템등에 사용됨  
  
이미지 검색시에는 각 차원은 피쳐의 특정공간 위치와 1:1로 대응이 됩니다.  
이런 데이터에서는 "좌표 간 거리"가 의미있는 물리적 차이\(형태, 색상 등\)을 나타내기에 조금더 정확할 수 있습니다.  
또한 백터의 크기도 이미지 밝기 등의 차이를 의미하기 때문에 코사인 유사도 방법보다 조금 더 이미지의 차이를 나타나기에 적합합니다.  
쉽게 이해하려면 아래처럼 해도 됩니다.    
* 문서와 같이 방향이 중요하면 코사인 유사도  
* 이미지와 같이 크기와 위치가 중요하면 유클리드 거리  

## 1.2.3 벡터 저장소  
"벡터 저장소" = "백터를 저장하고 검색할 수 있는 데이터 베이스"  
유사한 문서를 빠르게 찾는데 최적화 되어 있습니다.  
백터 저장소의 종류는 다음과 같습니다.  
  
*FAISS*
* 한줄 소개: 메타에서 개발한 오픈소스 검색 라이브러리입니다.  
* 장점: 다양한 인덱싱방식과 gpu를 사용하여 빠른 연산을 지원합니다.  
* 잔점: 정확도를 올리기 위해서는 튜닝이 필요하며, 메모리 사용량이 많으며, 분산환경 지원이 제한적 입니다.  
* 사례: 텍스트 검색, 이미지 검색  

*Chroma*
* 한줄 소개: JSON API제공 그리고 랭체인과 완벽한 통합을 지원하는 가볍고 빠른 벡터 저장소 입니다.  
* 장점: 속도가 빠릅니다. 랭체인과의 통합성이 좋습니다.  
* 단점: 대용량 데이터 처리와 분산처리 성능이 FAISS, Milvus보다 조금 부족합니다.  
* 사례: RAG, 문서검색  

*Milvus*  
* 한줄 소개: SQL like 대형/정밀한 벡터 데이터베이스 입니다.  
* 장점: 다량의 데이터에 적합하며 분산환경과 확장성이 좋습니다.  
* 단점: 운영이나 설정에 러닝커브가 있습니다.  
* 사례: 대규모 벡터 검색, 추천 시스템  
  
*Weaviate*  
* 한줄 소개: 다양한 클라우드 환경을 지원하고 다양한 임베딩 모델을 지원하는 백터 데이터베이스 입니다.  
* 장점: 다양한 클라우드를 지원합니다.  
* 단점: 기존 벡터 데이터베이스들보다 성능이 약간 낮다고 합니다.  
* 사례: 문서검색  
  
*Pinecone*  
* 한줄 소개: 클라우드 기반 유료 벡터 저장소 입니다.  
* 장점: 완전 관리형 서비스로 유지보수 비용이 크게 없습니다.  
* 단점: 유료입니다.  
* 사례: 실시간 AI 검색  
  
## 1.2.4 VectorRAG란?  
"VectorRAG" = "RAG의 하나의 형태이며 벡터 검색 기술을 활용하여 AI 응답을 보강하는 방법"  
VectorRAG의 핵심은 임베딩과 벡터 저장소를 활용하여 검색하는 것입니다.  
  
## 1.2.4 VectorRAG는 언제 사용해야 하나요?  
* 적합: 비정형 데이터, 대량의 문서 데이터, 변화가 빠른 데이터, 도메인 특화 데이터, 멀티모달 데이터  
* 부적합: 구조화된 데이터  
단순한 키워드 검색이 아닌, 문맥을 이해하는 의미 검색이 필요한 환경에서 적극 사용됩니다.  

In [8]:
'''
임배딩을 생성하는 openai 임배딩 생성 코드
'''
from dotenv import load_dotenv
import openai
load_dotenv()

client = openai.OpenAI()
response = client.embeddings.create(
    input=["AI 기술의 발전 과정"], # 입력은 리스트 형태로 해야합니다.
    model="text-embedding-ada-002",
)

embedding = response.data[0].embedding
print(embedding[:5]) # 
print(type(embedding), type(embedding[0])) # List[Float]
print(len(embedding)) # 1536 차원의 임베딩

[-0.0018854063237085938, -0.023751001805067062, 0.014851200394332409, -0.006142500322312117, -0.007527975365519524]
<class 'list'> <class 'float'>
1536


In [40]:
'''
FAISS를 사용하는 백터 검색 코드
'''
from langchain.vectorstores import FAISS
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from dotenv import load_dotenv
import os

llm = ChatOpenAI(
    model_name = "gpt-4o",
    temperature=0,
)

# 샘플 입력
sample_texts = [
    "AI 기술은 1950년대에 등장했고 이후에 계속 발전하였다.", 
    "트랜스포머 모델은 2017년 구글이 발표한 Attention All you need 논문에서 소개되었다.", 
    "chatGPT는 2022년 12월에 공개되었다.", 
    "FAISS는 Meta에서 개발하였다.", 
    "백터 검색 기술은 자연어뿐 아니라 이미지에서도 유용하게 사용하는 기술이다."
]

# 문서 Document 객체로 변환
documents = [Document(page_content=text, metadata={"doc_id": idx}) for idx, text in enumerate(sample_texts)] # 리스트의 인덱스를 메타로 넣어줌
# 문서분할(chunking)
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=50, chunk_overlap=10)
# docs = text_splitter.split_documents(documents)

# 모델 래디
embeddings = OpenAIEmbeddings()

# FAISS 백터 저장소 생성
# vectorstore = FAISS.from_documents(docs, embeddings)
vectorstore = FAISS.from_documents(documents, embeddings)

# 백터 검색을 위한 retriever 생성
retriever = vectorstore.as_retriever()

# 사용자의 질문을 기반으로 관련 문서 검색
query = "FAISS는 무엇인가?"
retrieved_docs = retriever.invoke(query)

for doc in retrieved_docs:
    print(doc)
print('-'*50)

top_k = 1
retrieved_docs = retrieved_docs[:top_k]

retrieved_text = "\n".join([doc.page_content for doc in retrieved_docs])
print(retrieved_text)
print('-'*50)

answer = llm.invoke(f"REFERENCE 내용을 기반으로 '{query}'에 대해서 답변하라 ## REFERENCE: {retrieved_text}")
print(answer.content)

page_content='FAISS는 Meta에서 개발하였다.' metadata={'doc_id': 3}
page_content='AI 기술은 1950년대에 등장했고 이후에 계속 발전하였다.' metadata={'doc_id': 0}
page_content='백터 검색 기술은 자연어뿐 아니라 이미지에서도 유용하게 사용하는 기술이다.' metadata={'doc_id': 4}
page_content='트랜스포머 모델은 2017년 구글이 발표한 Attention All you need 논문에서 소개되었다.' metadata={'doc_id': 1}
--------------------------------------------------
FAISS는 Meta에서 개발하였다.
--------------------------------------------------
FAISS는 Meta에서 개발한 라이브러리로, 대규모 벡터 검색과 유사도 검색을 효율적으로 수행하기 위해 설계되었습니다. 주로 고차원 벡터 공간에서의 최근접 이웃 검색을 빠르게 처리하는 데 사용됩니다. FAISS는 대량의 데이터셋을 다루는 머신러닝 및 인공지능 응용 프로그램에서 유용하게 활용됩니다.


In [ ]:
#TODO
'''
Chroma를 사용하는 백터 검색 코드
'''